<a href="https://colab.research.google.com/github/bec2148/mnist-finalproject/blob/main/MNIST_finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [553]:
from keras.datasets import mnist        # MNIST dataset is included in Keras
from keras.models import Sequential     # Model type to be used

from keras.layers import Dense, Dropout, Activation # Types of layers to be used in our model
from keras import utils                 # Keras utilities

In [554]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, Conv3D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Flatten
from tensorflow.keras.layers import BatchNormalization


In [555]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [556]:
# We do not flatten each image into a 784-length vector because we want to perform convolutions first

X_train = X_train.reshape(60000, 28, 28, 1) #add an additional dimension to represent the single-channel
X_test = X_test.reshape(10000, 28, 28, 1)

X_train = X_train.astype('float32')         # change integers to 32-bit floating point numbers
print(type(X_train))
print(type(X_test))
X_test = X_test.astype('float32')

X_train /= 255                              # normalize each value for each pixel for the entire vector for each input
X_test /= 255

print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Training matrix shape (60000, 28, 28, 1)
Testing matrix shape (10000, 28, 28, 1)


In [557]:
# padded_array = np.pad(array, ((1, 1), (0, 1)), mode='constant')
padded_X_train = np.pad(X_train, pad_width=((0, 0), (1, 1), (1, 1), (0, 0)), mode='constant', constant_values=0)
# ur = array[:, :] - padded_array[0:3, 1:4]

                                           # rows, cols
ur = X_train[:, :, :, :] - padded_X_train[:, 0:28, 2:30, :]
lr = X_train[:, :, :, :] - padded_X_train[:, 2:30, 2:30, :]

combined_X_train = np.concatenate((X_train, ur), axis=-1)
combined_X_train = np.concatenate((combined_X_train, lr), axis=-1)
combined_X_train = combined_X_train.reshape(60000, 28, 28, 3)

padded_X_test = np.pad(X_test, pad_width=((0, 0), (1, 1), (1, 1), (0, 0)), mode='constant', constant_values=0)

                                         # rows, cols
ur = X_test[:, :, :, :] - padded_X_test[:, 0:28, 2:30, :]
lr = X_test[:, :, :, :] - padded_X_test[:, 2:30, 2:30, :]

X_test = np.concatenate((X_test, ur), axis=-1)
X_test = np.concatenate((X_test, lr), axis=-1)
X_test = X_test.reshape(10000, 28, 28, 3)


Training matrix shape (60000, 28, 28, 3)
Training matrix shape (60000, 28, 28, 3)
Testing matrix shape (10000, 28, 28, 3)


In [558]:
# one-hot format classes

nb_classes = 10 # number of unique digits

Y_train = utils.to_categorical(y_train, nb_classes)
Y_test = utils.to_categorical(y_test, nb_classes)

In [559]:
model = Sequential()                                 # Linear stacking of layers

# Convolution Layer 1
model.add(Conv2D(16, (3, 3), input_shape=(28, 28, 3))) # 32 different 3x3 kernels -- so 32 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
convLayer01 = Activation('relu')                     # activation
model.add(convLayer01)

# Convolution Layer 2
model.add(Conv2D(32, (3, 3)))                        # 32 different 3x3 kernels -- so 32 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
model.add(Activation('relu'))                        # activation
convLayer02 = MaxPooling2D(pool_size=(2,2))          # Pool the max values over a 2x2 kernel
model.add(convLayer02)

# Convolution Layer 3
model.add(Conv2D(64,(3, 3)))                         # 64 different 3x3 kernels -- so 64 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
convLayer03 = Activation('relu')                     # activation
model.add(convLayer03)

# Convolution Layer 4
model.add(Conv2D(64, (3, 3)))                        # 64 different 3x3 kernels -- so 64 feature maps
model.add(BatchNormalization(axis=-1))               # normalize each feature map before activation
model.add(Activation('relu'))                        # activation
convLayer04 = MaxPooling2D(pool_size=(2,2))          # Pool the max values over a 2x2 kernel
model.add(convLayer04)
model.add(Flatten())                                 # Flatten final 4x4x64 output matrix into a 1024-length vector

# Fully Connected Layer 5
model.add(Dense(512))                                # 512 FCN nodes
model.add(BatchNormalization())                      # normalization
model.add(Activation('relu'))                        # activation

# Fully Connected Layer 6
model.add(Dropout(0.2))                              # 20% dropout of randomly selected nodes
model.add(Dense(10))                                 # final 10 FCN nodes
model.add(Activation('softmax'))                     # softmax activation

In [560]:
model.summary()

Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_104 (Conv2D)                  │ (None, 26, 26, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_142              │ (None, 26, 26, 16)          │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_170 (Activation)          │ (None, 26, 26, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_105 (Conv2D)                  │ (None, 24, 24, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_143              │ (None, 24, 24, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_171 (Activation)          │ (None, 24, 24, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_56 (MaxPooling2D)      │ (None, 12, 12, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_106 (Conv2D)                  │ (None, 10, 10, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_144              │ (None, 10, 10, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_172 (Activation)          │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_107 (Conv2D)                  │ (None, 8, 8, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_145              │ (None, 8, 8, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_173 (Activation)          │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_57 (MaxPooling2D)      │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_28 (Flatten)                 │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_56 (Dense)                     │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_146              │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_174 (Activation)          │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_28 (Dropout)                 │ (None, 512)                 │              

 Total params: 593,194 (2.26 MB)

 Trainable params: 591,818 (2.26 MB)

 Non-trainable params: 1,376 (5.38 KB)

In [561]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [563]:
model.fit(combined_X_train, Y_train, batch_size=128, epochs=3, verbose=1, validation_data=(X_test, Y_test))

Epoch 1/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 59s 101ms/step - accuracy: 0.9354 - loss: 0.2077 - val_accuracy: 0.9831 - val_loss: 0.0493
Epoch 2/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 48s 102ms/step - accuracy: 0.9916 - loss: 0.0281 - val_accuracy: 0.9837 - val_loss: 0.0492
Epoch 3/3
469/469 ━━━━━━━━━━━━━━━━━━━━ 62s 59ms/step - accuracy: 0.9953 - loss: 0.0171 - val_accuracy: 0.9908 - val_loss: 0.0301


In [564]:
score = model.evaluate(X_test, Y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9879 - loss: 0.0381
Test score: 0.030099572613835335
Test accuracy: 0.9908000230789185
